## 서울특별시 다산콜센터(☎120)의 주요 민원
* 서울특별시 다산콜센터(☎120)의 주요 민원(자주 묻는 질문)에 대한 답변정보
* https://opengov.seoul.go.kr/civilappeal/list

In [1]:
# 필요한 라이브러리들을 불러옵니다.
# pandas : 파이썬에서 사용할 수 있는 엑셀과 유사한 데이터분석 도구입니다.
# requests : 매우 작은 브라우저로 웹사이트의 내용과 정보를 불러옵니다.
# BeautifulSoup : request로 가져온 웹사이트의 html 태그를 찾기 위해 사용합니다.
# time : 한 번에 많은 양의 데이터를 가져오게 되면 서버에서 부담을주기 때문에 시간 간격을 두고 가져오기 위해 사용합니다.


import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup as bs
import re
import time

In [2]:
# 수집해온 파일("seoul-120-list.csv")을 읽어옵니다.
# df

df=pd.read_csv("seoul-120-list.csv")

## 일부 데이터를 우선 수집
* 현업에서는 모든 데이터를 수집해야 하지만 모든 데이터 수집시 서버에 부담이 될 수 있으며 오래 걸립니다.
* 또 동작하지 않거나 잘못 수집을 했다면 처음부터 다시 수집해야 하기 때문에 일부 데이터만을 통해 먼저 수집해 보는 것을 추천합니다.
* 일부 데이터를 먼저 수집해 보고 데이터가 필요할 때 전체를 수집해 봅니다.

In [31]:
# head 로 일부 데이터만 가져와서 먼저 수집해 봅니다.
df = df.head(10)

## 특정 내용 읽어오기
* 내용의 URL을 확인합니다.
* requests를 통해 내용에 접근합니다.
* BeautifulSoup의 select를 사용해 내용이 있는 태그를 찾습니다.


In [32]:
# 내용 페이지의 주소를 url 변수에 담아줍니다.
# 웹페이지의 결과를 받아옵니다.
# BeautifulSoup을 통해 lxml로 파싱해 올 수 있도록 합니다.
# html 태그에서 "div.line-all"의 0번째 값을 선택(select)하여 text를 확인합니다. 

url = "https://opengov.seoul.go.kr/civilappeal/view/?nid=23194045"
print(url)
# response
response = requests.get(url)
html = bs(response.text)
content = html.select('#content > div > div.view-content.view-content-article > div:nth-child(2) > div > p')[0].get_text()
content

content1 = html.select('#content > div > div.view-content.view-content-article > div:nth-child(3)')[0].get_text()

content1
# html
# content

https://opengov.seoul.go.kr/civilappeal/view/?nid=23194045


'\n문서 정보\n\n\n[서울산업진흥원] 서울메이드란? - 문서정보 : 원본시스템, 제공부서, 작성자(책임자), 생산일, 관리번호, 분류\n\n\n\n\n\n\n\n\n원본시스템\n다산콜센터\n\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t제공부서\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\n서울산업진흥원\n\n\n작성자(책임자)\n120다산콜재단\n생산일\n2021-06-29\n\n\n관리번호\nD0000042894548\n분류\n경제\n\n\n\n\n\n'

In [33]:
pd.read_html(response.text)[1]

,0,1,2,3
0,원본시스템,다산콜센터,제공부서,서울산업진흥원
1,작성자(책임자),120다산콜재단,생산일,2021-06-29
2,관리번호,D0000042894548,분류,경제


In [34]:
#content > div > div.view-content.view-content-article > div:nth-child(3) > div > table

table = pd.read_html(response.text)[1]
# table[[0,1]].set_index(0)

table



,0,1,2,3
0,원본시스템,다산콜센터,제공부서,서울산업진흥원
1,작성자(책임자),120다산콜재단,생산일,2021-06-29
2,관리번호,D0000042894548,분류,경제


In [35]:
t01 = table[[0,1]].set_index(0).T
t01



,원본시스템,작성자(책임자),관리번호
1,다산콜센터,120다산콜재단,D0000042894548


In [36]:
t02 = table[[2,3]].set_index(2).T
t02

2,제공부서,생산일,분류
3,서울산업진흥원,2021-06-29,경제


In [37]:
# 해당 문의가 어떤 분류에 해당되는지 알기 위해 분류를 수집합니다.
def get_desc(response):
    """ 분류 수집하기 """
    table = pd.read_html(response.text)[-1]
    t01 = table[[0,1]].set_index(0).T
    t02 = table[[2,3]].set_index(2).T
    t02.index = t01.index
    df_desc = pd.concat([t01,t02], axis=1)
    return df_desc


get_desc(response)

,원본시스템,작성자(책임자),관리번호,제공부서,생산일,분류
1,다산콜센터,120다산콜재단,D0000042894548,서울산업진흥원,2021-06-29,경제


## 내용 수집 함수 만들기

In [38]:
# 위의 전체 과정을 함수로 만들어 봅니다.

def get_view_page(view_no):
    """ 
    내용과 분류를 수집하는 함수 만들기
    """
    
    
    url = f"https://opengov.seoul.go.kr/civilappeal/view/?nid={view_no}"
    
    response = requests.get(url)
    html = bs(response.text)
    content = html.select('#content > div > div.view-content.view-content-article > div:nth-child(2) > div')
    #     desc = get_desc(response)
    
    if content : 
        content = content[0].get_text()
    else:
        content : ""
    df_desc = get_desc(response)
    df_desc["내용"] = content
    return df_desc

In [39]:
# 없는 데이터를 불러왔을 때 오류가 나는지 확인
# view_no가 22904492 인 것을 get_view_page()로 확인해 봅니다.
get_view_page(view_no=22904492)

,원본시스템,작성자(책임자),관리번호,제공부서,생산일,분류,내용
1,다산콜센터,120다산콜재단,D0000042557522,서울특별시 광진구 복지국 가정복지과,2021-05-13,복지,\n 업무개요 교통약자인 임산부와 영아가정 주민이 병원 진료를 받으러 갈 떄...


## 전체 내용 가져오기
* <font color="red">실습을 위해 위에서 일부 데이터만 가져왔습니다.</font>
* 전체 데이터 수집시에서 전체 수집한 데이터로 수집을 진행하면 됩니다.
* 전체 데이터 수집시에는 서버에 부담이 가지 않도록 time.sleep()을 통해 간격을 두고 수집해 주세요.

In [40]:
# tqdm.notebook 의 tqdm 을 통해 수집 진행상태를 확인합니다.
# progress_apply 를 사용하면 진행상태를 확인하며 데이터를 가져올 수 있습니다.
from tqdm.notebook import tqdm
tqdm.pandas()
view = df["내용번호"].progress_map(get_view_page)

## 수집한 내용 확인하기

In [41]:
# 일부 데이터만 슬라이싱하여 살펴봅니다.
view

0       원본시스템  작성자(책임자)            관리번호            ...
1       원본시스템  작성자(책임자)            관리번호     제공부서   ...
2       원본시스템  작성자(책임자)            관리번호            ...
3       원본시스템  작성자(책임자)            관리번호            ...
4       원본시스템  작성자(책임자)            관리번호            ...
5       원본시스템  작성자(책임자)            관리번호       제공부서 ...
6       원본시스템  작성자(책임자)            관리번호            ...
7       원본시스템  작성자(책임자)            관리번호      제공부서  ...
8       원본시스템  작성자(책임자)            관리번호            ...
9       원본시스템  작성자(책임자)            관리번호            ...
Name: 내용번호, dtype: object

In [43]:
# 하나만 가져오면 데이터프레임 형태로 되어 있음을 확인합니다.
view[4]

,원본시스템,작성자(책임자),관리번호,제공부서,생산일,분류,내용
1,다산콜센터,120다산콜재단,D0000040829456,서울특별시 여성가족정책실 여성권익담당관,2020-09-16,복지,\n 업무개요 직장 내 성희롱 지원체계 사각지대에 있는 서울 시민이 안전하...


## 하나로 병합하기

In [46]:
# 수집한 내용을 tolist() 를 통해 리스트로 변환 후 concat 으로 병합합니다.
df_view = pd.concat(view.tolist())


df_view.head()

,원본시스템,작성자(책임자),관리번호,제공부서,생산일,분류,내용
1,다산콜센터,120다산콜재단,D0000043274644,서울특별시 서초구 주민생활국 여성보육과,2021-08-17,복지,\n 업무개요 다자녀가정 아이들의 안전한 실내 활동과 층간소음 예방을 ...
1,다산콜센터,120다산콜재단,D0000042894548,서울산업진흥원,2021-06-29,경제,\n 서울의 감성을 담은 다양하고 새로운 경험을 제공하기 위해 만들어진 라이프스타일...
1,다산콜센터,120다산콜재단,D0000042557522,서울특별시 광진구 복지국 가정복지과,2021-05-13,복지,\n 업무개요 교통약자인 임산부와 영아가정 주민이 병원 진료를 받으러 갈 떄...
1,다산콜센터,120다산콜재단,D0000042114742,서울특별시 복지정책실 복지기획관 장애인복지정책과,2021-03-12,복지,\n ▣ 마포뇌병변장애인 비전센터 운영 구분 내용 목적 학령기 이...
1,다산콜센터,120다산콜재단,D0000040829456,서울특별시 여성가족정책실 여성권익담당관,2020-09-16,복지,\n 업무개요 직장 내 성희롱 지원체계 사각지대에 있는 서울 시민이 안전하...


In [45]:
# 기존 데이터와 병합하여 내용이 함께 수집된 것을 확인합니다.
df_detail

NameError: name 'df_detail' is not defined

## 사용할 컬럼만 남기기

In [ ]:
# 제목 뒤에 문서가 보이도록 컬럼의 순서를 조정합니다.
# 사용할 컬럼만 인덱싱 합니다.
# 다음의 순서가 되게 합니다. '번호', '분류', '제목', '내용', '내용번호'
df_detail

## 파일로 저장하고 확인하기

<img src="https://pandas.pydata.org/docs/_images/02_io_readwrite.svg">

In [ ]:
# 저장할 파일명을 "seoul-120-sample.csv" 로 설정해 줍니다.
file_name = "seoul-120-sample.csv"

In [ ]:
# csv 파일로 저장합니다.


In [ ]:
# 저장이 잘 되었는지 csv 파일로 읽어옵니다.
